In [1]:
import abcEconomics as abce
from abcEconomics import Simulation, Agent, Household, Firm
import numpy as np
import os
import pandas as pd
#import matplotlib.pyplot as plt
#%matplotlib inline

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
#   パラメータ群
N_HH = 10**3    #   家計の数
P = 1   #   物価水準
corporation_tax_rate = 0.2
consume_tax_rate = 0.1
devidend_rate = 0.1

In [ ]:
simulation = Simulation(name='simpleSFC', processes=1)

In [ ]:
class Household(Household, Agent):
    def init(self, family_name, potential_wage):
        self.family_name = family_name
        self.create("MS", 0)
        self.create("L", 0)
        self.wage   = max(0, potential_wage)  #   給料
        self.dividend_income = 0 #   受取配当の合計
        self.income_from_government = None  #   政府からの収入
        self.income = max(180, self.wage)  #   収入の合計
        self.income_log = np.zeros(5) + self.income
        self.net_lending = 0
        self.consumption = None
        self.potential_wage = potential_wage
        self.delta_L = 0
        self.delta_MS = 0
    
    def decide_wage_s1(self, wage_total_0, wage_total_1, potential_wage_SD):
        self.potential_wage *= wage_total_1/wage_total_0
        n = potential_wage_SD/7
        a = np.sqrt(potential_wage_SD**2 + n**2)
        self.potential_wage += n*np.random.randn()*(potential_wage_SD/a) + 2*potential_wage_SD*(a/potential_wage_SD - 1)
        if self.potential_wage <= 0:
            self.wage = 0
        else:
            self.wage = self.potential_wage
        return self.wage
    
    def decide_wage_s2(self, wage_total_1, wage_total_2):
        self.wage *= wage_total_1/wage_total_2
        
        

    def consume(self, mean_wage):
        center, width = 5*mean_wage, 5*mean_wage
        baseline, variable = 0.1, 0.5
        u = np.sum((self.income_log[:-1] - self.income_log[1:])/(np.arange(4) + 1))
        s = max(150, self.income + 4.0*u) + 0.1*self["MS"]
        t = (s - center)/width
        CR = baseline
        if t <= 10:
            CR += variable / (np.exp(t) + 1)
        self.consumption = CR*self.income / P
        self.buy(("firmC", 0), good='product', quantity=self.consumption, price=P, currency="MS")
        
    def print_possessions(self):
        print("     " + str(dict(self.possessions())))
    
    def pay_consume_and_income_tax(self, mean_wage):
        self.buy(("bank", 0), good="consume_tax", quantity=1, price=consume_tax_rate*self.consumption, currency="MS")
        t = np.exp((-self.wage+10*mean_wage)/(20*mean_wage))
        self.buy(("bank", 0), good="income_tax", quantity=1, price=0.5*t/(t + 1)*self.wage, currency="MS")
        self.tax = consume_tax_rate*self.consumption + 0.5*t/(t + 1)*self.wage
    
    def receive_wage(self):
        self.wage = 0
        for offer in self.get_offers("wage"):
            self.create("wage", 1)
            self.accept(offer)
            self.wage += offer.price

    def receive_dividend(self):
        self.dividend_income = 0
        for dividend_offer in self.get_offers("dividend"):
            self.dividend_income += dividend_offer.price
            self.create("dividend", 1)
            self.accept(dividend_offer)
    
    def receive_subsidy(self, bank):
        self.income_from_government = 0
        if (self.wage + self.dividend_income) < 180 and self["MS"] < 180:
            p = 180 - (self.wage + self.dividend_income)
            self.create("subsidy", 1)
            self.sell(("bank", 0), good="subsidy", quantity=1, price=p, currency="MS")
            bank.pay_households_subsidy()
            bank.receive_households_subsidy(p)
            self.income_from_government = 180 - (self.wage + self.dividend_income)
            self.income += 180 - (self.wage + self.dividend_income)
    
    def cal_net_lending(self):
        self.net_lending = self.income - self.consumption - self.tax
    
    def cal_income(self):
        self.income = self.wage + self.dividend_income + self.income_from_government
        self.income_log[1:] = self.income_log[:-1]
        self.income_log[0] = self.income
    
    def create_dividend_right(self):
        self.create("dividend", 1)

    def output_wage(self):
        return self.wage
    
    def output_potential_wage(self):
        return self.potential_wage
    
    def output_MS(self):
        return self["MS"]
    
    def reset_delta(self):
        self.delta_L = 0
        self.delta_MS = 0

    def output_income(self):
        return self.income
    
    def reset_fictitious_goods(self):
        self.destroy("consume_tax", self["consume_tax"])
        self.destroy("income_tax", self["income_tax"])
        self.destroy("wage", self["wage"])
        self.destroy("product", self["product"])
        
    
agent_params = [
    {
        "family_name":str(i), 
        "potential_wage":300*np.random.randn()+300, #######ここのパラメータは書き換える必要がある#######
    } for i in range(N_HH)
]
households = simulation.build_agents(Household, "household", agent_parameters=agent_params)


In [ ]:
class FirmC(Firm, Agent):
    def init(self, firm_name, wage_sum):
        self.firm_type = "typeC"
        self.firm_name = firm_name
        self.create("MS", 0)
        self.create("L", 0)
        self.create("product", 0)
        self.create("product_4_G", 0)
        self.profit = None  #   利潤
        self.profit_after_tax = None    #   税引後当期純利益
        self.dividend = None    #   配当
        self.undistributed_profit = None   #   内部留保
        self.net_lending = None
        self.wage_sum = wage_sum
        self.firm_income = None
        self.tax_sum = None
        self.delta_L = 0
        self.delta_MS = 0

    def cal_profit(self):
        self.profit = self.firm_income - self.wage_sum
    
    def cal_profit_after_tax(self):
        self.profit_after_tax = self.profit - self.tax_sum
        self.undistributed_profit = (1 - devidend_rate)*max(0, self.profit_after_tax)
        self.net_lending = self.undistributed_profit    #   実物資産がないのでそう言える
        if self.net_lending:
            if abs(self.delta_MS - self.delta_L - self.net_lending) > 1:
                print("企業のフローの会計的一貫性が崩れている", self.delta_MS - self.delta_L - self.net_lending)
    
    def pay_corporation_tax(self):
        #   いったん銀行に支払って、銀行が政府に支払う
        if self.profit <= 0:
            self.tax_sum = 0
            return None
        self.buy(("bank", 0), good="corporation_tax", quantity=1, price=corporation_tax_rate*self.profit, currency="MS")
        self.tax_sum = corporation_tax_rate*self.profit
    
    def sell_to_government(self):
        for offer in self.get_offers("product_4_G"):
            self.create("product_4_G", offer.quantity)  #   受注した分をすべて生産する
            self.accept(offer)
            self.firm_income += offer.price * offer.quantity

    def sell_to_households(self):
        self.firm_income = 0
        for offer in self.get_offers("product"):
            self.create("product", offer.quantity)  #   受注した分をすべて生産する
            self.accept(offer)
            self.firm_income += offer.price * offer.quantity
        
    def print_possessions(self):
        print("     " + str(dict(self.possessions())))

    def input_wage_sum(self, households):
        self.wage_sum = sum(households.output_wage()[0])
    
    def add_loan(self, price, bank):
        self.create("L", price)
        self.create("MS", price)
        bank.create_loan(price)
        self.delta_L += price

    def return_loan(self, bank):
        if self["L"] * self["MS"] != 0: #   self["L"] >= 0, self["MS"] >= 0
            price = min(self["L"], self["MS"])
            self.destroy("L", price)
            self.destroy("MS", price)
            bank.destroy_loan(price)
            self.delta_L -= price

            
    def pay_wage_1(self, households, bank, nominal_wage_growth_rate=0):
        wage_total_0 = self.wage_sum
        wage_total_1 = (1 + nominal_wage_growth_rate)*wage_total_0
        potential_wage_SD = np.var(households.output_potential_wage()[0])**0.5
        households.decide_wage_s1(wage_total_0, wage_total_1, potential_wage_SD)
        wage_total_2 = sum(households.output_wage()[0])
        households.decide_wage_s2(wage_total_1, wage_total_2)
        self.wage_sum = wage_total_1
        self.add_loan(self.wage_sum, bank)  #   賃金の支払いをいったん全額借り入れで賄う。コードを簡単にするため。

    def pay_wage_2(self, households):
        for household_id in range(len(households)):
            wage_price = households[household_id].output_wage()[0][0]
            self.buy(("household", household_id), good="wage", quantity=1, price=wage_price, currency="MS")

    def create_product(self, quantity_of_product):
        self.create("product", quantity_of_product)

    def cal_dividend_weight(self, households):
        w = np.array(households["MS"])
        w[w<0.1*np.mean(w)] = 0
        w[np.random.rand(len(w))<0.1] = 0
        return w
    
    def pay_dividend(self, households, bank):
        dividend_sum = max(0, devidend_rate*self.profit_after_tax)
        self.dividend = dividend_sum
        w_arr = np.array(households.output_MS()[0])
        w_arr[w_arr<0] = 0
        w_arr[w_arr<0.1*np.mean(w_arr)] = 0
        w_arr[np.random.rand(len(w_arr))>0.1] = 0
        if np.sum(w_arr) == 0:
            return None
        w_arr /= np.sum(w_arr)
        self.add_loan(1 + dividend_sum, bank)   #   少し多めに借り入れる、5行下のMSが2*10^(-10)程度不足しています的なエラーを回避するため
        bank.create_loan(1 + dividend_sum)
        for i, w in enumerate(w_arr):
            if w != 0:
                households[i].create_dividend_right()
                self.buy(("household", i), good="dividend", quantity=w*dividend_sum, price=1, currency="MS")
        self.return_loan(bank)
    
    def say_income(self):
        return self.firm_income
    
    def reset_fictitious_goods(self):
        self.destroy("dividend", self["dividend"])
        self.destroy("corporation_tax", self["corporation_tax"])
        self.destroy("wage", self["wage"])
    
    def reset_delta(self):
        self.delta_L = 0
        self.delta_MS = 0

        
agent_params = [
    {
        "firm_name":0,
        "wage_sum":np.sum(households.output_wage()[0]),
    }
]
firmC = simulation.build_agents(FirmC, 'firmC', agent_parameters=agent_params)


In [ ]:
class Government(Agent):
    def init(self):
        self.create("HPM", 0)
        self.net_lending = None
        self.expenditure = None
        self.tax = None
        self.delta_HPM = 0
    
    def receive_tax(self):
        self.consume_tax = 0
        for offer in self.get_offers("consume_tax"):
            self.create("consume_tax", 1)
            self.accept(offer)
            self.destroy("HPM", offer.price)
            self.delta_HPM -= offer.price
            self.consume_tax += offer.price * offer.quantity
        self.income_tax = 0
        for offer in self.get_offers("income_tax"):
            self.create("income_tax", 1)
            self.accept(offer)
            self.destroy("HPM", offer.price)
            self.delta_HPM -= offer.price
            self.income_tax += offer.price * offer.quantity
        self.corporation_tax = 0
        for offer in self.get_offers("corporation_tax"):
            self.create("corporation_tax", 1)
            self.accept(offer)
            self.destroy("HPM", offer.price)
            self.delta_HPM -= offer.price
            self.corporation_tax += offer.price * offer.quantity
        self.tax = self.consume_tax + self.income_tax + self.corporation_tax

    def pay_gov_exp_2_firm(self):
        self.create("HPM", 300*N_HH*P)
        self.delta_HPM += 300*N_HH*P
        self.buy(("bank", 0), good="product_4_G", quantity=300*N_HH, price=P, currency="HPM")
        self.expenditure = 300*N_HH*P

    def pay_gov_exp_2_household(self):
        for subsidy_offer in self.get_offers("subsidy"):
            self.create("HPM", subsidy_offer.price)
            self.delta_HPM += subsidy_offer.price
            self.accept(subsidy_offer)
            self.expenditure += subsidy_offer.price * subsidy_offer.quantity
    
    def cal_net_lending(self):
        self.net_lending = self.expenditure - self.tax
    
    def reset_delta(self):
        self.delta_HPM = 0
    
    def reset_fictitious_goods(self):
        self.destroy("subsidy", self["subsidy"])
        self.destroy("consume_tax", self["consume_tax"])
        self.destroy("income_tax", self["income_tax"])
        self.destroy("corporation_tax", self["corporation_tax"])
        self.destroy("product_4_G", self["product_4_G"])
    

government = simulation.build_agents(Government, "government", number=1)


In [ ]:
class Bank(Agent):
    def init(self):
        self.create("L", 0)
        self.create("MS", 0)
        self.create("HPM", 0)
        self.delta_L = 0
        self.delta_MS = 0
        self.delta_HPM = 0
    
    def pay_corporation_tax(self, firmC):
        price = corporation_tax_rate*firmC.say_income()[0][0]
        self.buy(("government", 0), good="corporation_tax", quantity=1, price=price, currency="HPM")
        
    def receive_corporation_tax(self):
        for corporation_tax_offer in self.get_offers("corporation_tax"):
            self.accept(corporation_tax_offer)
    
    def receive_gov_exp_2_firm(self):
        self.gov_exp_2_firm = 0
        for offer in self.get_offers("product_4_G"):
            self.create("product_4_G", offer.quantity)
            self.accept(offer)
            self.gov_exp_2_firm += offer.price

    def pay_gov_exp_2_firm(self):
        self.buy(("firmC", 0), "product_4_G", quantity=1, price=self.gov_exp_2_firm, currency="MS")

    def receive_households_subsidy(self, p):
        self.create("subsidy", 1)
        self.sell(("government", 0), "subsidy", quantity=1, price=p, currency="HPM")

    def pay_households_subsidy(self):
        for subsidy_offer in self.get_offers("subsidy"):
            self.accept(subsidy_offer)

    def receive_consume_and_income_tax(self):
        self.consume_tax = 0
        for offer in self.get_offers("consume_tax"):
            self.create("consume_tax", 1)
            self.accept(offer)
            self.consume_tax += offer.price * offer.quantity
        self.income_tax = 0
        for offer in self.get_offers("income_tax"):
            self.create("income_tax", 1)
            self.accept(offer)
            self.income_tax += offer.price * offer.quantity

    def pay_consume_and_income_tax(self):
        self.buy(("government", 0), good="consume_tax", quantity=1, price=self.consume_tax, currency="HPM")
        self.buy(("government", 0), good="income_tax", quantity=1, price=self.income_tax, currency="HPM")
    
    def create_loan(self, price):
        self.create("L", price)
        self.create("MS", price)
        self.delta_L += price
        self.delta_MS += price
    
    def destroy_loan(self, price):
        self.destroy("L", price)
        self.destroy("MS", price)
        self.delta_L -= price
        self.delta_MS -= price
    
    def reset_delta(self):
        if abs(self.delta_L + self.delta_HPM - self.delta_MS) > 1:
            print("銀行の資金フローの整合性が取れていない")
        self.delta_L = 0
        self.delta_HPM = 0
        self.delta_MS = 0
    
    def reset_fictitious_goods(self):
        self.destroy("subsidy", self["subsidy"])
        self.destroy("consume_tax", self["consume_tax"])
        self.destroy("income_tax", self["income_tax"])
        self.destroy("product_4_G", self["product_4_G"])
        self.destroy("product", self["product"])

bank = simulation.build_agents(Bank, "bank", number=1)


In [ ]:
for r in range(10):
    print("round", r)
    simulation.advance_round(r)
    
    mean_wage = np.average(households.output_wage()[0])
    #   消費支出    需要主導のため、最初に実行するのがよいとの判断
        #   家計消費支出
    households.consume(mean_wage)
        #   政府消費支出
    government.pay_gov_exp_2_firm()
    bank.receive_gov_exp_2_firm()
    bank.pay_gov_exp_2_firm()

    #   企業の生産と売り上げ
    firmC.sell_to_households()
    firmC.sell_to_government()
    
    #   賃金の支払い
    firmC.pay_wage_1(households, bank)  #   households側の要求する賃金水準の更新、賃金と同額を借り入れる
    firmC.pay_wage_2(households)    #   賃金の振り込みオファー
    households.receive_wage()   #   賃金の受け取り
    firmC.return_loan(bank)     #   賃金用の借り入れのうち、返済で霧分を全額返済

    #   政府から家計への、低所得者への給付金
    households.receive_subsidy(bank)
    government.pay_gov_exp_2_household()

    #   所得税と消費税の支払い
    mean_wage = np.average(households.output_wage()[0])
    households.pay_consume_and_income_tax(mean_wage)
    bank.receive_consume_and_income_tax()
    bank.pay_consume_and_income_tax()

    #   企業の借入の返済
    firmC.return_loan(bank)

    #   企業の税引き前当期純利益の計算
    firmC.cal_profit()

    #   法人税の計算
    firmC.pay_corporation_tax()
    bank.receive_corporation_tax()
    bank.pay_corporation_tax(firmC)
    government.receive_tax()

    #   税引き後当期純利益の計算と、配当の支払い
    firmC.cal_profit_after_tax()
    firmC.pay_dividend(households, bank)
    households.receive_dividend()

    #   家計の収入の集計
    households.cal_income()

    #   家計と政府の純貸付の計算、企業の純貸付は粗利益の処理の時に計算済み
    households.cal_net_lending()
    government.cal_net_lending()
    
    households.panel_log(goods=["MS", "L"])
    households.agg_log(variables=[
        "wage", "dividend_income", "income_from_government", "income", 
        "net_lending", "consumption", "delta_L", "delta_MS"
    ])
    firmC.panel_log(goods=['MS', "L", "product", "product_4_G"])
    firmC.agg_log(variables=[
        "profit", "profit_after_tax", "dividend", "undistributed_profit", 
        "net_lending", "wage_sum", "firm_income", "tax_sum", "delta_L", "delta_MS",
    ])
    government.panel_log(goods=["HPM"])
    government.agg_log(variables=[
        "net_lending", "expenditure", "tax", "delta_HPM",
    ])
    bank.panel_log(goods=["L", "MS", "HPM"])
    bank.agg_log(variables=["delta_L", "delta_MS", "delta_HPM"])

    #   資金フローのリセット
    households.reset_delta()
    bank.reset_delta()
    firmC.reset_delta()
    government.reset_delta()

    #   仮想商品の削除
    households.reset_fictitious_goods()
    bank.reset_fictitious_goods()
    firmC.reset_fictitious_goods()
    government.reset_fictitious_goods()

simulation.finalize()


round 0
3238034.0145513527 3238034.0145513485


TypeError: unsupported operand type(s) for /: 'Action' and 'int'

simulation.finalize() must be specified at the end of simulation
simulation.finalize() must be specified at the end of simulation


In [ ]:
print(os.listdir(simulation.path))

['aggregated_firmC.csv',
 'aggregated_household.csv',
 'description.txt',
 'panel_firmC.csv',
 'panel_household.csv']

In [ ]:
panel_firmC_df = pd.read_csv(simulation.path + '/panel_firmC.csv')
panel_firmC_df.head()

,index,money,productC,net_asset,round,name
0,1,100.0,1,100,0,household99
1,2,100.0,1,100,0,household44
2,3,100.0,1,100,0,household41
3,4,100.0,1,100,0,household53
4,5,100.0,1,100,0,household50


In [ ]:
panel_firmC_df.pivot(index='round', columns='name', values='L').plot()

,index,money,productC,net_asset,round,name
0,1,100.0,1,100,0,household99
1,2,100.0,1,100,0,household44
2,3,100.0,1,100,0,household41
3,4,100.0,1,100,0,household53
4,5,100.0,1,100,0,household50


In [ ]:
aggregate_firmC_df = pd.read_csv(simulation.path + '/aggregate_firmC.csv')
print(aggregate_firmC_df.head())

name,household0,household1,household10,household11,household12,household13,household14,household15,household16,household17,...,household90,household91,household92,household93,household94,household95,household96,household97,household98,household99
round,,,,,,,,,,,,,,,,,,,,,
0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
1,100.5,98.0,101.0,98.0,98.5,98.0,102.0,100.5,98.0,98.5,...,101.0,98.5,98.0,98.5,100.5,101.0,101.0,98.0,98.0,102.0
2,99.5,98.0,101.5,98.0,99.5,98.0,101.5,99.5,98.0,99.5,...,101.5,99.5,98.0,99.5,99.5,101.5,101.5,98.0,98.0,101.5
3,100.0,98.5,100.5,98.5,99.0,98.5,102.0,100.0,98.5,99.0,...,100.5,99.0,98.5,99.0,100.0,100.5,100.5,98.5,98.5,102.0
4,99.0,98.5,100.5,98.5,101.0,98.5,101.0,99.0,98.5,101.0,...,100.5,101.0,98.5,101.0,99.0,100.5,100.5,98.5,98.5,101.0
5,99.5,94.5,105.0,94.5,98.5,94.5,102.5,99.5,94.5,98.5,...,105.0,98.5,94.5,98.5,99.5,105.0,105.0,94.5,94.5,102.5
6,99.5,94.0,105.5,94.0,99.0,94.0,102.0,99.5,94.0,99.0,...,105.5,99.0,94.0,99.0,99.5,105.5,105.5,94.0,94.0,102.0
7,101.0,93.0,105.5,93.0,96.0,93.0,104.5,101.0,93.0,96.0,...,105.5,96.0,93.0,96.0,101.0,105.5,105.5,93.0,93.0,104.5
8,101.5,93.0,103.0,93.0,95.5,93.0,107.0,101.5,93.0,95.5,...,103.0,95.5,93.0,95.5,101.5,103.0,103.0,93.0,93.0,107.0


In [ ]:
aggregate_firmC_df.pivot(index='round', columns='name', values='delta_L').plot()

name,firmC0,firmC1,firmC2,firmC3,firmC4
round,,,,,
0,78,83,84,79,76
1,57,61,64,61,57
2,39,42,43,40,36
3,30,26,30,32,32
4,21,35,38,29,27
5,29,29,31,30,31
6,30,36,32,27,25
7,35,31,30,29,25
8,19,31,31,34,35


Customer offers 10 dollar:
    drug_dealer{'drugs': 1.0}
    customer{'MS': 90.0}
Drug Dealer accepts or rejects the offer:
    drug_dealer{'drugs': 1.0}
    customer{'MS': 90.0}

Customer offers 10 dollar:
    drug_dealer{'drugs': 0.0, 'MS': 10.0}
    customer{'MS': 80.0, 'drugs': 1.0}
Drug Dealer accepts or rejects the offer:
    drug_dealer{'drugs': 0.0, 'MS': 10.0}
    customer{'MS': 80.0, 'drugs': 1.0}



simulation.finalize() must be specified at the end of simulation
